In [728]:
using LinearAlgebra

function shrinkage(w,v)
    z = w-v
    if z >0
        return z
    else
        return 0
    end
end

function sparsity(a)
    if a != 0
        return 1
    else
        return 0
    end
end

# maskedM is the partially masked matrix to complete
# maxIterations is the maximum number of iterations
# tol is the tolerance (used in stopping conditions)
function matrixCompletion(maskedM, maxIterations = 300, tol = .0002,  𝜇 = 1, 𝛾 = 1.618)
    #Initialize Parameters
    n = size(maskedM,1)
    𝜌 = 2.5/n
    X = rand(Normal(0,1),n,n)
    # Make X symmetric so that it has real eigenvalues
    X = .5(X+X')
    #X = X/norm(X,2)
    Y = rand(Normal(0,1),n,n)
    #Y = Y/norm(Y,2)
    # Make Y symmetric
    Y = .5(Y+Y')
    Π = zeros(n,n)
    
    stop1 = zeros(maxIterations)
    stop2 = zeros(maxIterations)
    stop3 = zeros(maxIterations)
    
    #mulitplying by these elementwise is the projection onto the subspace of matrices with nonzeros 
    #on the sets Ω and Ω̂ respectively
    Ω = sparsity.(maskedM)
    Ω̂ = 1 .- Ω
    
    #Main Iteration
    for i in 1:maxIterations
        G = Y - (1/𝜌)*Π
        w,U = eigen(G)
        
        X_prev = deepcopy(X)
        X = U*Diagonal(shrinkage.(w,𝜇/𝜌))*U'
        
        E = X + (1/𝜌)*Π
        Y = (1/(𝜌+1)) * Ω .* (maskedM + 𝜌 * E) + Ω̂ .* E
        
        Π = Π + 𝛾 * 𝜌 * (X - Y)

        𝜇 = 𝜇/1.01
        #Check Stopping Conditions
        #X is not updating much
        #X is very close to Y
        #X is very close on the masked entries to the masked matrix
        
        stop1[i] = norm(X_prev-X, 2)
        stop2[i] = norm(X-Y, 2)
        stop3[i] = norm(Ω .* X - maskedM, 2)
        if stop1[i] <= tol && stop2[i] <= tol && stop3[i] <= tol
            break
        end  
    end
    return (X, stop1, stop2, stop3)
end

matrixCompletion (generic function with 5 methods)

In [712]:
using Distributions 
using PyPlot
using LinearAlgebra 

function experiment(m,r, 𝜎, SR, 𝜇)
    ML = rand(Normal(0,1),m,r)
    M  = ML*ML'
    Σ = Symmetric(rand(Normal(0,1),m,m))
    M_tilde = M + 𝜎 * Σ
    Ω = Symmetric(rand(Bernoulli(SR),m,m))
    X,stop1,stop2,stop3 = matrixCompletion(mask .* M_tilde)
    R = rank(real(X))
    Err = norm(X-M,2)
    return(Err,R)
end

experiment (generic function with 1 method)

In [660]:
# Code from a julia notebook for EECS 551 at Umich
# https://web.eecs.umich.edu/~fessler/course/551/julia/demo/09_lr_complete3.html

# Define cost function for optimization problem
nucnorm = (X) -> sum(svdvals(X))
costfun = (X,Y,Ω,beta) -> 0.5 * norm(X[Ω]-Y[Ω])^2 + beta * nucnorm(X)

#Define singular value soft thresholding (SVST)
SVST = (X,beta) -> begin
    U,s,V = svd(X)
    sthresh = max.(s .- beta,0)
    return U * Diagonal(sthresh) * V'
end;

# Apply ISTA (Iterative Soft-Thresholding Algorithm)
niter = 400
beta = 0.01 # chosen by trial-and-error here
function lrmc_ista(Y)
	X = copy(Y)
	Xold = copy(X)
	cost_ista = zeros(niter+1)
	cost_ista[1] = costfun(X,Y,beta)
	for k=1:niter
    	X[Ω] = Y[Ω]
    	X = SVST(X,beta)
    	cost_ista[k+1] = costfun(X,Y,beta)
	end
	return X, cost_ista
end

# Run FISTA algorithm
niter = 200
# beta = 0.01
function lrmc_fista(Y,Ω)
	X = copy(Y)
	Z = copy(X)
	Xold = copy(X)
	told = 1
	cost_fista = zeros(niter+1)
	cost_fista[1] = costfun(X,Y,Ω,beta)
	for k=1:niter
    	Z[Ω] = Y[Ω]
    	X = SVST(Z,beta)
    	t = (1 + sqrt(1+4*told^2))/2
    	Z = X + ((told-1)/t)*(X-Xold)
    	Xold = X
    	told = t
    	cost_fista[k+1] = costfun(X,Y,Ω,beta) # comment out to speed-up
	end
	return X, cost_fista
end


#function fista_experiment(m,r, 𝜎, SR, 𝜇)
function fista_experiment(m,r, 𝜎, SR, 𝜇)
    ML = rand(Normal(0,1),m,r)
    M  = ML*ML'
    Σ = Symmetric(rand(Normal(0,1),m,m))
    M_tilde = M + 𝜎 * Σ
    Omega = Symmetric(rand(Bernoulli(SR),m,m))
    Ω = Omega .!= 0
    X,cost_fista = lrmc_fista(M_tilde, Ω)
    R = rank(real(X))
    Err = norm(X-M,2)
    return(Err,R)
end

fista_experiment (generic function with 1 method)

In [735]:
using Plots
using LaTeXStrings
x = 1:300

Plots.plot(x,[log.(stop1),log.(stop2),log.(stop3)], title = "Stopping Conditions", label = [L"||X_k-X_{k-1}||" L"||X_k-Y_k||" L"||X_\Omega - M_\Omega||"],lw = 3,show = true)
Plots.savefig("Stopping Conditions")
#Plots.plot!(log.(stop2))
#Plots.plot!(log.(stop3))

In [730]:
sigma = 0
r = 5
n = 100
SR = .45
print(experiment(n,r,sigma,SR,1))
print(fista_experiment(n,r,sigma,SR,1))

(0.1650839200118543, 5)(0.06777503985239013, 5)

In [731]:
sigma = 0.001
r = 5
n = 100
SR = .45
print(experiment(n,r,sigma,SR,1))
print(fista_experiment(n,r,sigma,SR,1))

(0.1793456908056056, 5)(0.09277425557547524, 16)

In [732]:
sigma = .01
r = 5
n = 100
SR = .45
print(experiment(n,r,sigma,SR,1))
print(fista_experiment(n,r,sigma,SR,1))

(0.7150000117687665, 29)(1.0336770757398177, 50)

In [733]:
sigma = .1
r = 5
n = 100
SR = .45
print(experiment(n,r,sigma,SR,1))
print(fista_experiment(n,r,sigma,SR,1))

(12.30039282685151, 46)(11.226976946648406, 54)

In [734]:
sigma = .3
r = 5
n = 100
SR = .45
print(experiment(n,r,sigma,SR,1))
print(fista_experiment(n,r,sigma,SR,1))

(47.211772782816595, 50)(32.78378457723613, 56)